In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import time
import ssl
import os


In [ ]:
#For ignoring SSL ceritficate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [ ]:
#helper function get first number in string

def str_first_num(s):
    first_num = ''
    flag_digit=False
    for el in s: 
        if el.isdigit():
            first_num += el
            flag_digit = True
        else:
            if flag_digit:
                break
    return first_num
    

In [ ]:
#read question urls from csv file
src_path = r"C:\Users\Ravit\Documents\horizon_scanning_lab\Scrapers\Quora_RCS\answers_and_info\quet_indexes.csv"
df = pd.read_csv(src_path)

In [ ]:
#get list of urls
url_list = list(df['url'].values)

In [ ]:
url_list

In [ ]:
#request url html
page = requests.get(url_list[0])

#create soup object with htmal parser
soup = BeautifulSoup(page.content, 'html.parser')


In [ ]:
answers = soup.find_all('script', {"type": "application/ld+json"})[0]
#convert soup object to string
answers = [str(a) for a in answers][0]
#split answers with text separator
answers_list = answers.split('"text":')

In [ ]:
columns = ["Answer", "name", "knowsAbout",  "hasCredential", "upvoteCount", "affiliation", "answerCount", "followerCount"]

#create df with nans to fill it later with values
x_shape = (len(answers_list)-1, len(columns))
x = np.tile(np.nan, x_shape)
answers_df = pd.DataFrame(x, columns = columns)

In [ ]:
#get answer value
split_post = answers_list[1].split("upvoteCount")
answers_df["Answer"].iloc[0] = split_post[0]

#remove answer
split_post = split_post[1]

#get upvote value
upvote = str_first_num(split_post)

answers_df["upvoteCount"].iloc[0] = float(upvote)

#get affiliation which is right before answercount
split_post = split_post.split("affiliation")
split_post = split_post[1]
split_post = split_post.split("answerCount")

answers_df["affiliation"].iloc[0] = split_post[0]

split_post = split_post[1]

#get answer count
answers_df["answerCount"].iloc[0] = str_first_num(split_post)

#get followercount
split_post = split_post.split("followerCount")
answers_df['followerCount'] .iloc[0]= str_first_num(split_post[1])

split_post = split_post[1]

#get person credentials (between hascredentials and knowsabout)
split_post = split_post.split("hasCredential")[1]
split_post = split_post.split("knowsAbout")

#remove url from credential part
answers_df['hasCredential'].iloc[0] = split_post[0].split("image")[0]
split_post = split_post[1]

#Finally, get knowsabout and name. Remove url image from knowsabout too
split_post = split_post.split("name")
answers_df['knowsAbout'].iloc[0] = split_post[0].split("image")[0]

answers_df['name'].iloc[0] = split_post[1]


 

In [ ]:
answers_list[1]

In [ ]:
def answers_into_df(url):
    '''input: url for scrapping answers to a particular question.
        out: df with answers for each url/question'''
    
    #request url html
    page = requests.get(url)
   
    #create soup object with htmal parser
    soup = BeautifulSoup(page.content, 'html.parser')

    answers = soup.find_all('script', {"type": "application/ld+json"})[0]
    #convert soup object to string
    answers = [str(a) for a in answers][0]
    #split answers with text separator
    answers_list = answers.split('"text":')
    #first element is just the header of a page. remove it
    answers_list = answers_list[1:]
    
    #create df with nans to fill it later with values
    columns = ["Answer", "upvoteCount", "affiliation", "answerCount", "followerCount", "hasCredential",  "knowsAbout", "name"]
     
    x_shape = (len(answers_list), len(columns))
    x = np.tile(np.nan, x_shape)
    answers_df = pd.DataFrame(x, columns = columns)

    for i, ans in enumerate(answers_list):    
        #get answer value
        split_post = ans.split("upvoteCount")
        answers_df["Answer"].iloc[i] = split_post[0]

        #remove answer
        split_post = split_post[1]

        #get upvote value
        upvote = str_first_num(split_post)

        answers_df["upvoteCount"].iloc[i] = float(upvote)

        #get list of categories that exist in answer post
        existing_columns = [col for col in columns if col in ans]
                
        #get values to columns that exist in post
        
        for c in existing_columns:          
            if len(split_post)>1:
                split_post = split_post[1]
            try:
                split_post = split_post.split(c)
            except:
                pass
            #if category is numeric get number
            if "Count" in c:
                answers_df[c].iloc[i] = str_first_num(split_post) 
            #remove first element of split
            if len(split_post)>1:
                split_post = split_post[1]
    return answers_df 
        
        
        
        

            

In [104]:

answers_into_df(url_list[0])

In [ ]:
    '''   
                split_post = split_post.split("affiliation")
                split_post = split_post[1]
                split_post = split_post.split("answerCount")

                answers_df["affiliation"].iloc[i] = split_post[0]

                split_post = split_post[1]

                #get answer count
                answers_df["answerCount"].iloc[i] = str_first_num(split_post)

                #get followercount
                split_post = split_post.split("followerCount")
                answers_df['followerCount'] .iloc[i]= str_first_num(split_post[1])

                split_post = split_post[1]

                #get person credentials (between hascredentials and knowsabout)
                split_post = split_post.split("hasCredential")[1]
                split_post = split_post.split("knowsAbout")

                #remove url from credential part
                answers_df['hasCredential'].iloc[i] = split_post[0].split("image")[0]
                split_post = split_post[1]

                #Finally, get knowsabout and name. Remove url image from knowsabout too
                split_post = split_post.split("name")
                answers_df['knowsAbout'].iloc[i] = split_post[0].split("image")[0]

                answers_df['name'].iloc[i] = split_post[1]

        return answers_df   
        '''  

In [ ]:
columns

In [ ]:
ex_it = iter(columns)

In [ ]:
next(ex_it)